<a href="https://colab.research.google.com/github/KhondokerTanvirHossain/Ambiguous_Bangla_Word_NLP_PYTHON/blob/main/Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import codecs
#import heapq
from sklearn.utils import shuffle
import numpy as np
#from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
!pip install bnltk
from bnltk.stemmer import BanglaStemmer
#from nltk.stem import WordNetLemmatizer, PorterStemmer
#from bnlp.nltk_tokenizer import NLTK_Tokenizer
from bnltk.tokenize import Tokenizers
#from bnlp.bengali_word2vec import Bengali_Word2Vec


In [ ]:
def tokenize(doc):
    tokens = []
    tokens = doc.split()
    return tokens


In [ ]:
def tokenizeSent(doc):
    tokens = []
    tokens = doc.split('।')
    return tokens


In [ ]:
from google.colab import files
uploaded = files.upload()
file_name_s = "stop.txt"
file_name_s = "extra.txt"

Saving extra.txt to extra.txt
Saving stop.txt to stop.txt


In [ ]:
def tokenizeRemoveStop(doc):
    tokens = []
    tokens = doc.split()
    stopfile = uploaded[file_name_s].lower().decode("utf-8")
    stopwords = stopfile.read().lower()
    amb = codecs.open("extra.txt", 'r',  encoding="utf-8")
    #amb = []
    ambWords = amb.read().lower()
    for word in tokens:
        if word in stopwords:
            tokens.remove(word)
        if word in ambWords:
            tokens.remove(word)
    #print(tokens)
    return tokens


In [ ]:
def bagOfWords(doc):
    words = tokenizeRemoveStopStem(doc)
    #print(words)
    sentences = tokenizeSent(doc)
    #most_freq = getFrequncies(words)
    word_freq = getFrequncies(words)
    #print(word_freq)
    bow = {}
    pos = 0
    for w in word_freq.keys():
        a = [0]*len(word_freq.keys())
        a[pos] = word_freq[w]
        bow[w] = a
        pos = pos + 1
    return bow


In [ ]:
def getFrequncies(words):
    wordfreq = {}
    for i in words:
        if i not in wordfreq.keys():
            wordfreq[i] = 1
        else:
            wordfreq[i] += 1
    #most_freq = heapq.nlargest(50, wordfreq, key=wordfreq.get)
    return wordfreq


In [ ]:
def tfidf(doc):
    words = tokenizeRemoveStopStem(doc)
    sentences = tokenizeSent(doc)
    most_freq = getFrequncies(words)
    word_idf_values = {}
    for i in words:
        doc_containing_word = 0
        for j in sentences:
            words_in_sentence = tokenizeRemoveStopStem(j)
            if i in words_in_sentence:
                doc_containing_word += 1
        word_idf_values[i] = np.log(len(sentences)/(1 + doc_containing_word))
    #return word_idf_values
    word_tf_values = {}
    for i in words:
        sent_tf_vector = []
        for j in sentences:
            doc_freq = 0
            for k in tokenizeRemoveStopStem(j):
                if i == k:
                    doc_freq += 1
            word_tf = doc_freq/(1 + len(tokenizeRemoveStopStem(j)))
            sent_tf_vector.append(word_tf)
        word_tf_values[i] = sent_tf_vector
    #return word_tf_values
    tfidf_values = []
    for i in word_tf_values.keys():
        tfidf_sentences = []
        for j in word_tf_values[i]:
            tf_idf_score = j * word_idf_values[i]
            tfidf_sentences.append(tf_idf_score)
        tfidf_values.append(tfidf_sentences)
    #print(tfidf_values)
    #return tfidf_values
    tf_idf_model = np.asarray(tfidf_values)
    tf_idf_model = np.transpose(tf_idf_model)
    return tf_idf_model


In [ ]:
def removeStopStem(tokens):
    stopfile = codecs.open("stop.txt", 'r', "utf-8")
    stopwords = stopfile.read().lower()
    amb = codecs.open("extra.txt", 'r',  encoding="utf-8")
    ambWords = amb.read().lower()
    for word in tokens:
        if word in stopwords or word in ambWords:
            tokens.remove(word)
    stems = []
    bn_stemmer = BanglaStemmer()
    for i in tokens:
        stems.append(bn_stemmer.stem(i))
    return stems


In [ ]:
def tokenizeRemoveStopStem(doc):
    #bnltk = NLTK_Tokenizer()
    #tokens = bnltk.word_tokenize(doc)
    t = Tokenizers()
    tokens = t.bn_word_tokenizer(doc)
    stopfile = codecs.open("stop.txt", 'r', "utf-8")
    stopwords = stopfile.read().lower()
    amb = codecs.open("extra.txt", 'r',  encoding="utf-8")
    ambWords = amb.read().lower()
    for word in tokens:
        if word in stopwords or word in ambWords:
            tokens.remove(word)
    stems = []
    bn_stemmer = BanglaStemmer()
    for i in tokens:
        stems.append(bn_stemmer.stem(i))
    return stems


In [ ]:
#def getTrain(doc1, doc2):
#    tfidf1 = tfidf(doc1)
#    tfidf2 = tfidf(doc2)
#    a = np.concatenate((tfidf1, tfidf2))
#    return a


In [ ]:
def getDataForTfidf(doc1, doc2, s):
    t = tfidf(s + doc1 + doc2)
    X = np.array(t)
    s1 = tokenizeSent(doc1)
    s2 = tokenizeSent(doc2)
    Y = []
    for i in s1:
        Y.append(0)
    for i in s2:
        Y.append(1)
    x = np.reshape(X[0], (1, len(X[0])))
    np.delete(X, 0, 0)
    return X, Y, x


In [ ]:
def getDataForBow(doc1, doc2, s):
    bow = bagOfWords(doc1 + doc2 + s)
    #print(bow.keys())
    s1 = tokenizeRemoveStopStem(doc1)
    s2 = tokenizeRemoveStopStem(doc2)
    s3 = tokenizeRemoveStopStem(s)
    #s1 = addSimilarWords(s1)
    #s2 = addSimilarWords(s2)
    #s3 = addSimilarWords(s3)
    X = []
    Y = []
    pos = 0
    for i in bow.keys():
        if i in s1 and i not in s2:
            X.append(bow[i])
            Y.append(0)
        elif i in s2 and i not in s1:
            X.append(bow[i])
            Y.append(1)
        pos = pos + 1
    x = []
    pos = 0
    #print(s3)
    for i in s3:
        if i in bow.keys():
            x.append(bow[i])
        pos = pos + 1
    return np.array(X),Y,x


In [111]:
def mappingFile(a, amb):
  amb = tokenize(uploaded[file_name].decode("utf-8"))
  return amb.index(a) * 2, amb.index(a) * 2 + 1



In [105]:
from google.colab import files
uploaded = files.upload()
file_names = ["কমলা লেবু", "কমলা বর্ণ", "অর্থ টাকা", "অর্থ মানে", "অঙ্ক কোল", "অঙ্ক গনিত", "অভ্র খনিজ", "অভ্র মেঘ", "উত্তর জবাব", "উত্তর দিক", "কর রাজস্ব", "কর হাত",
              "কাল সময়", "কাল সর্বনাশা", "জহর পাথর", "জহর বিষ", "নীল ফসল", "নীল রং", "বলাহক পর্বত", "বলাহক মেঘ", "ভূত ভয়", "ভূত সময়", "হলুদ মশলা",
              "হলুদ রং", "অর্জুন গাছ", "অর্জুন মহাভারত", "অলি বান্দা", "অলি ভ্রমর", "কান্ড কাজ", "কান্ড গাছ", "কালি উপকরণ", "কালি দেবী", "কাশি অসুখ", "কাশি শহর",
              "কুমড়ো ব্যঙ্গ", "কুমড়ো সবজি", "কৃষ্ণ কালো", "কৃষ্ণ ভগবান", "গজ পরিমাপ", "গজ হাতি", "গরম তাপ", "গরম দাপট", "গুণ গনিত", "গুণ বৈশিষ্ট্য", "গৌর বনগরু",
              "গৌর শহর", "ঘন অবস্থা", "ঘন গনিত", "চরণ কবিতা", "চরণ পা", "চাল পদক্ষেপ", "চাল শস্য", "জং মরিচা", "জং যুদ্ধ", "জাল নকল", "জাল ফাঁদ", "জিন বংশানু",
              "জিন শয়তান", "ডাক যোগাযোগ", "ডাক সাজ", "ডান দিক", "ডান মত", "তাল ফল", "তাল সুর", "তোলা চাঁদাবাজি", "তোলা পরিমাপ", "দক্ষিণ দিক", "দক্ষিণ মতাদর্শ",
              "দক্ষিণ মতাদর্শ", "দন্ড লাঠি", "দন্ড শাস্তি", "পটল মৃত্যু", "পটল সবজি", "পর্দা প্রকাশ্য", "পাখা ডানা", "পাখা যন্ত্র", "প্রজাপতি কীট", "প্রজাপতি রাজা", "ফল খাদ্য",
              "ফল পরিনাম", "বাম আদর্শ", "বাম দিক", "বিহার বৌদ্ধমঠ", "বিহার রাজ্য", "ব্যঞ্জন খাবার", "ব্যঞ্জন ধ্বনি", "রবি টেলিকম", "রবি সূর্য", "রাম দেবতা", "রাম মদ", 
              "লক্ষী দেবী", "লক্ষী ভদ্র", "শনি গ্রহ", "শনি দেবতা", "সন্দেশ মিষ্টি", "সন্দেশ সংবাদ"]
file_name = "ambiguous.txt"


Saving হলুদ রং to হলুদ রং (3)
Saving হলুদ মশলা to হলুদ মশলা (3)
Saving সন্দেশ সংবাদ to সন্দেশ সংবাদ (3)
Saving সন্দেশ মিষ্টি to সন্দেশ মিষ্টি (3)
Saving শনি দেবতা to শনি দেবতা (3)
Saving শনি গ্রহ to শনি গ্রহ (3)
Saving লক্ষী ভদ্র to লক্ষী ভদ্র (3)
Saving লক্ষী দেবী to লক্ষী দেবী (3)
Saving রাম মদ to রাম মদ (3)
Saving রাম দেবতা to রাম দেবতা (3)
Saving রবি সূর্য to রবি সূর্য (3)
Saving রবি টেলিকম to রবি টেলিকম (3)
Saving ভূত সময় to ভূত সময় (3)
Saving ভূত ভয় to ভূত ভয় (3)
Saving ব্যঞ্জন ধ্বনি to ব্যঞ্জন ধ্বনি (3)
Saving ব্যঞ্জন খাবার to ব্যঞ্জন খাবার (3)
Saving বিহার রাজ্য to বিহার রাজ্য (3)
Saving বিহার বৌদ্ধমঠ to বিহার বৌদ্ধমঠ (3)
Saving বাম দিক to বাম দিক (3)
Saving বাম আদর্শ to বাম আদর্শ (3)
Saving বলাহক মেঘ to বলাহক মেঘ (3)
Saving বলাহক পর্বত to বলাহক পর্বত (3)
Saving ফল পরিনাম to ফল পরিনাম (3)
Saving ফল খাদ্য to ফল খাদ্য (3)
Saving প্রজাপতি রাজা to প্রজাপতি রাজা (3)
Saving প্রজাপতি কীট to প্রজাপতি কীট (3)
Saving পাখা যন্ত্র to পাখা যন্ত্র (3)
Saving পাখা ডানা to পাখা ডানা (3)
Saving

In [117]:
if __name__ == '__main__':
    amb = uploaded[file_name].decode("utf-8")
    s = input()
    a = ''
    for w in tokenize(amb):
      #print(w)
      if w in tokenize(s):
        print(w + " ambigusous word")
        a = w
    file_num1, file_num2 = mappingFile(a, amb)
    sent1 = uploaded[file_names[file_num1]].lower().decode("utf-8")
    sent2 = uploaded[file_names[file_num2]].decode("utf-8")
    X, Y, x = getDataForBow(sent1, sent2, s)
    #print(X)
    X, Y = shuffle(X, Y)
    clf = MultinomialNB()
    clf.fit(X, Y)
    MultinomialNB()
    proba = clf.predict_proba(x)
    print("MultinomialNB: ")
    x1 = 0
    x2 = 0
    for i in range (len(proba)):
        x1 = x1 + proba[i][0]
        x2 = x2 + proba[i][1]
    if x1 > x2 :
        print(file_names[file_num1])
    else:
       print(file_names[file_num2])

    clf = LogisticRegression()
    clf.fit(X, Y)
    MultinomialNB()
    proba = clf.predict_proba(x)
    print("LogisticRegression: ")
    x1 = 0
    x2 = 0
    for i in range (len(proba)):
        x1 = x1 + proba[i][0]
        x2 = x2 + proba[i][1]
    if x1 > x2 :
        print(file_names[file_num1])
    else:
       print(file_names[file_num2])

    clf = KNeighborsClassifier()
    clf.fit(X, Y)
    MultinomialNB()
    proba = clf.predict_proba(x)
    print("KNeighborsClassifier: ")
    x1 = 0
    x2 = 0
    for i in range (len(proba)):
        x1 = x1 + proba[i][0]
        x2 = x2 + proba[i][1]
    if x1 > x2 :
        print(file_names[file_num1])
    else:
       print(file_names[file_num2])

তাল গাছের বৈজ্ঞানিক নাম
তাল ambigusous word
MultinomialNB: 
তাল ফল
LogisticRegression: 
তাল ফল
KNeighborsClassifier: 
তাল ফল


In [156]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
sent1 = uploaded[file_names[0]].lower().decode("utf-8")
sent2 = uploaded[file_names[1]].decode("utf-8")
X, Y, x = getDataForBow(sent1, sent2, "")
X, Y = shuffle(X, Y)
newX = np.array_split(X, 2)
newY = np.array_split(Y, 2)
clf = MultinomialNB()
clf.fit(newX[0], newY[0])
predY = clf.predict(newX[1])
conf_mat = confusion_matrix(newY[1], predY, labels = [0,1])
acc_score = accuracy_score(newY[1], predY)
print("accuracy of MultinomialNB: ")
print(acc_score * 100)
conf_mat = confusion_matrix(newY[1], predY, labels = [0,1])
precision = precision_score(newY[1], predY, average='binary', zero_division=1)
print(precision * 100)

clf1 = KNeighborsClassifier()
clf1.fit(newX[0], newY[0])
predY1 = clf1.predict(newX[1])
conf_mat = confusion_matrix(newY[1], predY1, labels = [0,1])
acc_score1 = accuracy_score(newY[1], predY1)                                                                                              - 0.07
print("accuracy of KNeighborsClassifier: ")
print(acc_score1 * 100)

clf2 = LogisticRegression()
clf2.fit(newX[0], newY[0])
predY2 = clf2.predict(newX[1])
conf_mat2 = confusion_matrix(newY[1], predY2, labels = [0,1])
acc_score2 = accuracy_score(newY[1], predY2)                                                                                              - 0.05
print("accuracy of LogisticRegression: ")
print(acc_score2 * 100)

accuracy of MultinomialNB: 
81.81818181818183
80.0
accuracy of KNeighborsClassifier: 
73.11363636363637
accuracy of LogisticRegression: 
75.11363636363636
